In [92]:
import pandas as pd
import numpy as np
import geopandas as gpd
import folium, fiona, branca

In [93]:
df = pd.read_csv('../data/clean/deliveries.csv')

In [94]:
df = df[["id", "pickupLat", "pickupLng", "deliveryLat", "deliveryLng"]]

In [95]:
df

,id,pickupLat,pickupLng,deliveryLat,deliveryLng
0,t-amabrt-1743660,-19.9618,-44.0285,-19.4522,-44.2543
1,t-amabrt-1743661,-19.9618,-44.0285,-19.8932,-43.9991
2,t-amabrt-1743662,-19.9618,-44.0285,-19.8820,-44.0004
3,t-amabrt-1743663,-19.9618,-44.0285,-19.7904,-43.9844
4,t-amabrt-1743664,-19.9618,-44.0285,-19.8798,-43.9029
...,...,...,...,...,...
90149,t-petz-3242007,-19.9592,-44.1852,-19.9422,-44.1731
90150,t-petz-3242056,-19.9627,-43.9570,-19.9804,-43.9667
90151,t-petz-3242066,-19.9524,-43.9388,-19.9354,-43.9375
90152,t-petz-3242068,-19.9524,-43.9388,-19.9291,-43.9234


In [96]:
gdf_deliveries = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df["deliveryLng"], df["deliveryLat"]))

In [97]:
gdfBH = gpd.read_file('../data/BH_regions.geojson', driver='GeoJSON')

In [98]:
gdfBH.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [99]:
gdf_deliveries.crs = gdfBH.crs

In [100]:
dfgeo = gpd.sjoin(gdf_deliveries, gdfBH, how='inner', op='within')

c:\Python39\lib\site-packages\IPython\core\interactiveshell.py:3377: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [101]:
dfgeo

,id,pickupLat,pickupLng,deliveryLat,deliveryLng,geometry,index_right,ID,CODIGO,SIGLA,NOME,AREA_KM2,PERIMETR_M
1,t-amabrt-1743661,-19.9618,-44.0285,-19.8932,-43.9991,POINT (-43.99910 -19.89320),8,8,25,PA,PAMPULHA,51.038,37945.233
2,t-amabrt-1743662,-19.9618,-44.0285,-19.8820,-44.0004,POINT (-44.00040 -19.88200),8,8,25,PA,PAMPULHA,51.038,37945.233
12,t-amabrt-1743672,-19.9618,-44.0285,-19.8733,-43.9880,POINT (-43.98800 -19.87330),8,8,25,PA,PAMPULHA,51.038,37945.233
18,t-amabrt-1743678,-19.9618,-44.0285,-19.8726,-43.9819,POINT (-43.98190 -19.87260),8,8,25,PA,PAMPULHA,51.038,37945.233
22,t-amabrt-1743682,-19.9618,-44.0285,-19.8363,-44.0141,POINT (-44.01410 -19.83630),8,8,25,PA,PAMPULHA,51.038,37945.233
...,...,...,...,...,...,...,...,...,...,...,...,...,...
90001,t-petz-3239725,-19.9515,-44.0267,-19.8263,-43.9490,POINT (-43.94900 -19.82630),0,6,23,NT,NORTE,32.555,34762.823
90008,t-petz-3239777,-19.8586,-43.9591,-19.8041,-43.9479,POINT (-43.94790 -19.80410),0,6,23,NT,NORTE,32.555,34762.823
90042,t-petz-3239889,-19.8586,-43.9591,-19.8002,-43.9457,POINT (-43.94570 -19.80020),0,6,23,NT,NORTE,32.555,34762.823
90054,t-petz-3239948,-19.8586,-43.9591,-19.8278,-43.9582,POINT (-43.95820 -19.82780),0,6,23,NT,NORTE,32.555,34762.823


In [102]:
gdfgeo = dfgeo.groupby('NOME')["id"].count().reset_index()

In [103]:
gdfgeo = gdfBH.merge(gdfgeo, on='NOME')

In [104]:
gdfgeo.columns = ['zone_id', 'zone_code', 'sigla', 'nombre', 'area_km2', 'perimeter_km', 'geometry', 'n_deliveries']

In [105]:
gdfgeo.head()

,zone_id,zone_code,sigla,nombre,area_km2,perimeter_km,geometry,n_deliveries
0,6,23,NT,NORTE,32.555,34762.823,"POLYGON ((-43.94346 -19.79781, -43.94347 -19.7...",2270
1,4,21,NE,NORDESTE,39.321,40951.45,"POLYGON ((-43.87674 -19.81314, -43.87689 -19.8...",4911
2,3,20,LE,LESTE,27.885,31621.064,"POLYGON ((-43.90983 -19.87636, -43.90984 -19.8...",4419
3,2,19,CS,CENTRO-SUL,31.741,29872.501,"POLYGON ((-43.89457 -19.94749, -43.89458 -19.9...",17601
4,9,26,VN,VENDA NOVA,29.172,28995.848,"POLYGON ((-44.01057 -19.82590, -44.01057 -19.8...",2948


In [106]:
gdfgeo.to_file('../output/geo/deliveries_per_zone.geojson', driver='GeoJSON')

In [107]:
centroid=gdfBH.geometry.centroid
mapbox_api_key = r"pk.eyJ1Ijoic3RhbWVuIiwiYSI6IlpkZEtuS1EifQ.jiH_c9ShtBwtqH9RdG40mw"
tileset_ID_str = "streets-v11"
tilesize_pixels = "256"
m=folium.Map(
  location=[centroid.y.mean(), centroid.x.mean()],
  zoom_start=12,
  tiles = f"https://api.mapbox.com/styles/v1/mapbox/{tileset_ID_str}/tiles/{tilesize_pixels}/{{z}}/{{x}}/{{y}}@2x?access_token={mapbox_api_key}",
  attr='MAPBOX'
)

C:\Users\COREI7~1\AppData\Local\Temp/ipykernel_12992/1301057516.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid=gdfBH.geometry.centroid


In [108]:
field = "n_deliveries"
label = "Cantidad de deliveries por zona administrativa"
colormap = branca.colormap.linear.YlGn_07.to_step(6).scale(gdfgeo[field].min(),gdfgeo[field].max())
colormap.caption = label
layer = folium.GeoJson('../output/geo/deliveries_per_zone.geojson',
        name=label,
        style_function=lambda x: {"weight":1, 'color':'black','fillColor':colormap(x['properties'][field]), 'fillOpacity':1},
        highlight_function=lambda x: {'weight': 3, 'color':'black'},
        smooth_factor=2.0,
        tooltip=folium.features.GeoJsonTooltip(
          fields=['nombre',field],
          aliases=['nombre',label], 
          labels=True, 
          sticky=True,
          localize=True
        ),
        show = True,
        overlay = True
      )
m.add_child(layer)
m.add_child(colormap)

In [109]:
m.save('../output/choropleth.html')